# Tarea 1 - Redes densas
## Eduardo García Alarcón

### 2. Retropropagación en red densa

Programa el algoritmo de retropropagación usando NumPy para una tarea de clasificación binaria presuponiendo una red densa con dos capas ocultas. Las neuronas de las capas ocultas cuentan con una función de activasión ReLU, definida por $$ ReLU(x) = máx(0,z)$$

Por su parte, la capa de salida está compuesta por una sola neurona logística.

Para el entrenamiento minimiza el promedio de la función de pérdida de entropía cruzada binaria: 
$$
ECB(\bold{y}, \bold{\hat y}) = -\frac{1}{n} \sum_{i=1}^n \big[ y^{(i)} log \big(\hat y^{(i)}\big) + \big(1-y^{(i)}\big) log\big(1-\hat y^{(i)}\big)\big]
$$

Entrena la red mediante descenso por gradiente y el algoritmo de retropropagación de errores.

Describe las fórmulas y reglas de actualización de los pesos y sesgos de cada capa y entrena y evalúa la red en algún problema de clasifiación no lineal. Compara el comportamiento del entrenamiento de esta red con una en la que las neuronas de las capas ocultas tienen una función de activasión logística y en la que la función de pérdida no se promedia. 

In [2]:
# Bilbiotecas
import numpy as np

### Definimos la función de ReLU y su derivada

In [4]:
def relu(z):
  return np.max([0.0, z])

In [3]:
def derivada_relu(x):
  if x < 0.0:
    return 0.0
  else:
    return 1

De Igual manera la sigmoide

In [7]:
def sigmoide(z):
  return 1 / (1 + np.exp(-z))

In [ ]:
def derivada_sigmoide(x):
  s = sigmoide(x)
  return s * (1.0 - s)

### Definimos la función de Entropía Cruzada Binaria

In [5]:
def entropia_cruzada_binaria(y, p):
  # Para evitar divisiones entre 0
  p[p == 0] = np.nextafter(0., 1.)
  p[p == 1] = np.nextafter(1., 0.)
  
  return -( np.log( p[y==1] ).sum() + np.log(1 - p[y==0]).sum() )

#### Y definimos al exactitud

In [6]:
def exactitud(y, y_hat):
  return (y == y_hat).mean() * 100

#### Propagación